# Using _egobox_ optimizer _Egor_

## Installation

In [1]:
%pip install egobox

Note: you may need to restart the kernel to use updated packages.


We import _egobox_ as _egx_ for short and setup the logging to get optimization progress during the execution

In [2]:
import numpy as np
import egobox as egx

# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.DEBUG)

## Example 1 : Continuous optimization

### Test functions

In [3]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [4]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Constinuous optimization with _Egor_

In [5]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=1e-3,
                     infill_strategy=egx.InfillStrategy.WB2,
                     target=-5.5,
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Specify regression and/or correlation models used to build the surrogate
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [6]:
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor_solver:Compute initial LHS with 10 points
DEBUG:egobox_ego.egor_solver:INITIAL STATE = EgorState { param: None, prev_param: None, best_param: None, prev_best_param: None, cost: None, prev_cost: None, best_cost: None, prev_best_cost: None, target_cost: -5.5, iter: 0, last_best_iter: 0, max_iters: 30, counts: {}, time: Some(0ns), termination_status: NotTerminated, doe_size: 10, added: 10, prev_added: 0, no_point_added_retries: 3, lhs_optim: false, clusterings: Some([None, None, None]), data: Some(([[1.6652148798078232, 0.41440936777987125],
 [0.43156870256785784, 2.294326006943828],
 [1.166403281075166, 1.9625441958834666],
 [2.277840945398902, 3.449304213124792],
 [2.607296292826505, 2.875257061748758],
 [2.867876107405225, 1.3621098036060335],
 [2.0968198286824133, 0.8410733420421433],
 [1.2025125600954896, 0.19752068707849518],
 [0.12821018257976532, 2.7356141146526727],
 [0.8036513169756913, 3.880164026505513]], shape=[10, 2], strides=[2, 1], layout=Cc (0x5), con

Optimization f=[-5.5082731e+00 -1.2013287e-03  8.4967834e-04] at [2.32967959 3.17859352]


## Example 2 : Mixed integer optimization

### Test function

In [9]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType(egx.XType.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [10]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor_solver:Compute initial LHS with 3 points
DEBUG:egobox_ego.egor_solver:INITIAL STATE = EgorState { param: None, prev_param: None, best_param: None, prev_best_param: None, cost: None, prev_cost: None, best_cost: None, prev_best_cost: None, target_cost: -15.12, iter: 0, last_best_iter: 0, max_iters: 30, counts: {}, time: Some(0ns), termination_status: NotTerminated, doe_size: 3, added: 3, prev_added: 0, no_point_added_retries: 3, lhs_optim: false, clusterings: Some([None]), data: Some(([[20.928002989190862],
 [11.186676635671397],
 [1.931103999415601]], shape=[3, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2, [[-11.443706815463152],
 [5.135668199851883],
 [0.6892935156628215]], shape=[3, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2)), sampling: Some(Lhs { xlimits: [[0.0, 25.0]], shape=[1, 2], strides=[2, 1], layout=CFcf (0xf), const ndim=2, kind: Maximin, rng: RwLock { data: Xoshiro256Plus { s: [12052300764726007310, 17796918956915702363, 1249559064128353

Optimization f=[-15.12161154] at [19.]


## Usage

In [11]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, n_cstr=0, cstr_tol=1e-06, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, q_points=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (float):
 |       tolerance on constraints violation (cstr < tol).
 |  
 |   xspecs (list(XSpec)) where XSpec(xtype=FLOAT|INT, xlimits=[lower bound, upper bound]):
 |       Bounds of the nx compone